# Baseline model (Modèle de référence)

L'idée c'est de faire tourner un modèle simple afin d'avoir une référence pour comparer nos futurs modèles. A discuter de ce qu'on prend comme modèle de référence.

## Import (package + Train data frame)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
# from src.preprocessing import display_missing_values
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_theme(style="ticks", palette="pastel")

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("data/train.csv", sep= ",", low_memory=False,dtype={"ID":str})

## Preprocessing Train 

In [3]:
col_categoricals = df.select_dtypes(include="object").columns.tolist()

# Puis les numériques
col_numericals = [col for col in df.columns if col not in col_categoricals]

In [4]:
colonnes_a_drop = ["MMS","r","Enedc (g/km)","Ernedc (g/km)","Erwltp (g/km)","De","Vf"]
df_pp= df.drop(columns=colonnes_a_drop)
col_numericals_2 = [col for col in df_pp.columns if col not in col_categoricals]
df_pp= df_pp[col_numericals_2]

### Imputation des NaN 

In [5]:
df_pp.drop_duplicates(inplace=True)

In [6]:
mean_imputer= SimpleImputer(missing_values=np.nan, strategy='mean')

Impute par la moyenne ceux qu'on peut imputer par la moyenne

In [7]:
colonnes_mean_impute=['m (kg)','W (mm)','ep (KW)','At1 (mm)','At2 (mm)','Mt','ec (cm3)']
df_pp[colonnes_mean_impute]= mean_imputer.fit_transform(df_pp[colonnes_mean_impute])

Imputer par 0 car certaines variables correspondent à des critères seulement présent chez les voitures électriques. (ex: consommation electrique `z (Wh/km)` n'a pas lieu d'être sur une voiture essence)

In [8]:
fill_values = {'Fuel consumption ':0, 'z (Wh/km)': 0,'Electric range (km)':0}
df_pp.fillna(fill_values,inplace=True)

In [9]:
# display_missing_values(df_pp)

### Train Test Split (pour validation du modèle)

In [10]:
train, test = train_test_split(df_pp,test_size=0.33,random_state=42)

train.reset_index(drop=True, inplace=True ) #car ça fout la merde dans l'index
test.reset_index(drop = True, inplace = True)

In [11]:
X_train, y_train =train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test =test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

## Model (fit) + Validation

In [12]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

### Model predict pour observer la validation

In [13]:
# Prédire 
y_pred= model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 43.2842022687755


## Fit again sur Train entier (pour prédire sur Test)

In [14]:
X_train, y_train =df_pp.drop(columns=["Ewltp (g/km)"]), df_pp["Ewltp (g/km)"]

In [15]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

## Preprocessing Test

In [16]:
df_test = pd.read_csv("data/test.csv", sep= ",", low_memory=False,dtype={"ID":str})

In [17]:
colonnes_a_drop = ["MMS","r","Enedc (g/km)","Ernedc (g/km)","Erwltp (g/km)","De","Vf"]
df_pp_test= df_test.drop(columns=colonnes_a_drop)
col_numericals_2 = [col for col in df_pp_test.columns if col not in col_categoricals]
df_pp_test= df_pp_test[col_numericals_2]

### Imputation NaN

In [18]:
colonnes_mean_impute=['m (kg)','W (mm)','ep (KW)','At1 (mm)','At2 (mm)','Mt','ec (cm3)']
df_pp_test[colonnes_mean_impute]= mean_imputer.fit_transform(df_pp_test[colonnes_mean_impute])

In [19]:
fill_values = {'Fuel consumption ':0, 'z (Wh/km)': 0,'Electric range (km)':0}
df_pp_test.fillna(fill_values,inplace=True)

In [20]:
# display_missing_values(df_pp_test)

## Model Predict on Test

In [21]:
df_test["Ewltp (g/km)"] = model.predict(df_pp_test)

## Prepare for Submission

In [22]:
df_test["ID"]=df_test["ID"].apply(int)

In [23]:
df_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_lin_bateau.csv", index=False)

Modèle de régression polynomiale

In [24]:
from sklearn.preprocessing import PolynomialFeatures

In [29]:
train, test = train_test_split(df_pp,test_size=0.33,random_state=42)

train.reset_index(drop=True, inplace=True ) #car ça fout la merde dans l'index
test.reset_index(drop = True, inplace = True)

In [30]:
X_train, y_train =train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test =test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [31]:
polynome=PolynomialFeatures(degree=2,interaction_only=False)
X_train_polynome=polynome.fit_transform(X_train)
X_test_polynome=polynome.fit_transform(X_test)


In [32]:
model.fit(X_train_polynome,y_train)

LinearRegression()

In [33]:
y_pred_polynome=model.predict(X_test_polynome)

In [34]:
mean_absolute_error(y_test,y_pred_polynome)

29.677258262929392

In [35]:
X, y =df_pp.drop(columns=["Ewltp (g/km)"]), df_pp["Ewltp (g/km)"]

In [36]:
X_polynome=polynome.fit_transform(X)

In [37]:
model.fit(X_polynome,y)

LinearRegression()

In [39]:
X_true_test=polynome.fit_transform(df_pp_test)
df_test["Ewltp (g/km)"] = model.predict(X_true_test)

In [41]:
df_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_poly_bateau.csv", index=False)

#### Modèle de Regression Ridge

In [43]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

In [44]:
ridge = Ridge()

# Définissez l'intervalle d'alphas souhaité, par exemple, de 0.01 à 1000
alphas = np.logspace(-3, 2, 20)  # Crée 100 alphas entre 0.01 et 1000

# Utilisez make_scorer pour créer un scorer basé sur le MAE
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Utilisez GridSearchCV pour effectuer une recherche sur grille
param_grid = {'alpha': alphas}
grid_search = GridSearchCV(ridge, param_grid, scoring=mae_scorer, cv=5)  # cv est le nombre de plis dans la validation croisée

# Entraînez le modèle avec la recherche sur grille
grid_search.fit(X_train, y_train)

# Obtenez le meilleur modèle et les meilleurs hyperparamètres
best_model = grid_search.best_estimator_
best_alpha = grid_search.best_params_['alpha']

# Prédisez sur les données de test avec le meilleur modèle
y_pred = best_model.predict(X_test)

# Calculez le MAE sur les données de test
mae = mean_absolute_error(y_test, y_pred)

print(f"Meilleur modèle Ridge avec alpha={best_alpha}")
print(f"Mean Absolute Error sur les données de test: {mae:.3f}")


Meilleur modèle Ridge avec alpha=0.001
Mean Absolute Error sur les données de test: 43.284


In [45]:
ridge = Ridge()

# Définissez l'intervalle d'alphas souhaité, par exemple, de 0.01 à 1000
alphas = np.logspace(-3, 2, 20)  # Crée 100 alphas entre 0.01 et 1000

# Utilisez make_scorer pour créer un scorer basé sur le MAE
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Utilisez GridSearchCV pour effectuer une recherche sur grille
param_grid = {'alpha': alphas}
grid_search = GridSearchCV(ridge, param_grid, scoring=mae_scorer, cv=5)  # cv est le nombre de plis dans la validation croisée

# Entraînez le modèle avec la recherche sur grille
grid_search.fit(X_train_polynome, y_train)

# Obtenez le meilleur modèle et les meilleurs hyperparamètres
best_model = grid_search.best_estimator_
best_alpha = grid_search.best_params_['alpha']

# Prédisez sur les données de test avec le meilleur modèle
y_pred_polynome = best_model.predict(X_test_polynome)

# Calculez le MAE sur les données de test
mae = mean_absolute_error(y_test, y_pred_polynome)

print(f"Meilleur modèle Ridge avec alpha={best_alpha}")
print(f"Mean Absolute Error sur les données de test: {mae:.3f}")


c:\Users\jeann\Desktop\M2\New Data Mining Challenge\Projet_Mining\.venv_mining\lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.0978e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\jeann\Desktop\M2\New Data Mining Challenge\Projet_Mining\.venv_mining\lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.11036e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\jeann\Desktop\M2\New Data Mining Challenge\Projet_Mining\.venv_mining\lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.10739e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\jeann\Desktop\M2\New Data Mining Challenge\Projet_Mining\.venv_mining\lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning

Meilleur modèle Ridge avec alpha=0.001
Mean Absolute Error sur les données de test: 29.677
